<a href="https://colab.research.google.com/github/evaie/evaie/blob/main/process_reperes_dame_de_nages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get references data
Find Mattieu's file and read it into a dataframe


In [1]:
# piece of code that get a token
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate the user
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Once authenticated, list the files that contains personal records for the atheletes


In [16]:
# Auto-iterate through all files that matches this query
#file_list = drive.ListFile({'q': "'root' in parents and title contains 'Champs' and trashed=false"}).GetList()
#sharedWithMe=true 
# id = 1VJMXVNSxIv4li8yK7pXZ3sitWEdC8z-93z_Apkl77_Q
file_list = drive.ListFile({'q': "sharedWithMe=true and trashed=false and title contains '2021 - repères d'"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: 2021 - repères d'intensité, id: 1VJMXVNSxIv4li8yK7pXZ3sitWEdC8z-93z_Apkl77_Q


once the id of the spreadsheet retrieved, load it. 
But first, we need to be allowed Gspread  / GSheet to access to our data.

In [36]:
# piece of code that get a token
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Then load, the data into a dataframe.


In [23]:
sh = gc.open_by_key('1VJMXVNSxIv4li8yK7pXZ3sitWEdC8z-93z_Apkl77_Q')
worksheet = sh.get_worksheet(0)
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

import pandas as pd
df_reperes = pd.DataFrame.from_records(rows)

Reprocess the dataframe : renaming the columns


In [34]:
df_reperes
# column names un 1st row
# reformat
headers = df_reperes.iloc[0].apply(lambda x: str(x).replace(' ', '_'))
headers = headers.apply(lambda x: str(x).replace('(', ''))
headers = headers.apply(lambda x: str(x).replace(')', ''))
headers = headers.apply(lambda x: str(x).replace('.', ''))
headers = headers.apply(lambda x: str(x).replace('é', 'e'))
headers = headers.apply(lambda x: str(x).replace('_/_', '_'))
headers = headers.apply(lambda x: str(x).replace(',', '_'))
headers = headers.apply(lambda x: str(x).lower())

# rename columns
df_reperes = df_reperes.rename(columns=headers)
df_reperes = df_reperes.drop(0)
df_reperes

,nom,prenom,categorie,record_ergo,distance,sec_500,watts,objectif_watts_bateau_0_85,ftp,watts_b1,watts_b2
1,Boccanfuso,Inès,SFPL,"07:22,50",2000,"110,63",259,220,,132,165
2,Morizot,Aurélie,SFPL,"07:15,00",2000,"108,75",272,231,,139,173
3,Quenneville,Florian,SHTC,"06:16,50",2000,"94,13",420,357,,214,268
4,Gely,Axel,SHTC,"06:31,80",2000,"97,95",372,317,,190,237
5,De Villers,Louis,SHPL,"06:39,90",2000,"99,98",350,298,,179,223
6,Ferrière,Mathys,SHPL,"06:34,00",2000,"98,50",366,311,,187,233
7,Schwendenwein,Vincent,SHTC,"06:34,50",2000,"98,63",365,310,,186,233
8,Macre,Philippe,SHTC,"06:44,20",2000,"101,05",339,288,,173,216
9,Dore,Maxime,SHPL,"06:40,60",2000,"100,15",348,296,,178,222
10,Le Levreur,Jean,SHPL,"06:47,00",2000,"101,75",332,282,,169,212


# Get the data from the outriggers

In [4]:
import pandas as pd

# files whose title contains SpdCoach and not in the trash
file_list = drive.ListFile({'q': "title contains 'SpdCoach' and trashed=false"}).GetList()
ddd_list = list()
for ddd_file in file_list:
  # collect meta data from the filename
  filename = ddd_file['title']
  file_id = ddd_file['id']
  tokens = filename.split('-')
  rower = tokens[1].strip()
  training_type = tokens[2].split('.')[0].strip()
  date = tokens[0].split(' ')[2].strip() +':'+ tokens[0].split(' ')[3].strip()
  print('rower: %s, training type: %s, date: %s' % (rower, training_type, date))
  
  # this assume pyDrive has a token to read the GoogleDrive
  myfile = drive.CreateFile({'id': file_id})
  myfile.GetContentFile('file.csv')
  
  # then read it in a Dataframe
  ddd = pd.read_csv('file.csv', skiprows=27, header=None)

  # clean and organize the data
  # column names un 1st row
  # reformat
  headers = ddd.iloc[0].apply(lambda x: str(x).replace(' ', '_'))
  headers = headers.apply(lambda x: str(x).replace('(', ''))
  headers = headers.apply(lambda x: str(x).replace(')', ''))
  headers = headers.apply(lambda x: str(x).replace('.', ''))
  headers = headers.apply(lambda x: str(x).lower())

  # rename columns
  ddd = ddd.rename(columns=headers)

  # dropping NA values
  ddd2 = ddd.dropna()

  # drop the 2 first lines
  ddd2 = ddd2.drop([0, 1])

  # drop the 2 last rows
  #ddd2 = ddd2.drop(ddd2.tail(2).index)

  # reset the index
  ddd2 = ddd2.reset_index(drop=True)

  # changing column types
  column_names = list(ddd2.columns)

  # datetime columns
  # remove lines with date time columns formated to '---'
  ddd2 = ddd2.drop(ddd2[ddd2['elapsed_time'].map(lambda x: str(x) == '---')].index)
  ddd2 = ddd2.drop(ddd2[ddd2['split_gps'].map(lambda x: str(x) == '---')].index)
  ddd2 = ddd2.drop(ddd2[ddd2['split_imp'].map(lambda x: str(x) == '---')].index)

  # iterating on columns and change the types to string
  for col in column_names:
      ddd2[col] = ddd2[col].apply(lambda x: str(x).replace('---', '0'))

  ddd2 = ddd2.convert_dtypes()

  # removing datetime columns from the columns list
  column_names.pop(3)  # 'elapsed_time'
  column_names.pop(3)  # 'split_gps'
  column_names.pop(4)  # 'split_imp'

  # add meta data
  ddd2['rower'] = rower
  ddd2['training_type'] = training_type
  ddd2['date'] = date

  for col in column_names:
    ddd2[col] = pd.to_numeric(ddd2[col])

  # accumulate ddd
  ddd_list.append(ddd2)


rower: Adrien, training type: 500 et recup, date: 20210403:0814am
rower: Louis, training type: 500, date: 20210403:0814am
rower: Louis, training type: recup, date: 20210403:0824am
rower: Louis, training type: 1000, date: 20210331:0420pm
rower: Adrien, training type: 1000, date: 20210331:0420pm
rower: Yohan, training type: 1000, date: 20210331:0253pm


In [5]:
# concat dataframes
all_ddd = pd.concat(ddd_list)


Save the reperes worksheet

In [43]:
import gspread
import gspread_dataframe as gd

# this piece of code assumes we already have a token to read / write a Google Sheet
# 1e_ThufABmmmbUlqQ05VoCrXfChhDuQHMnZQMYw73NxA is the id of DameDeNage_database
db_sh = gc.open_by_key('1e_ThufABmmmbUlqQ05VoCrXfChhDuQHMnZQMYw73NxA')

wsheet = db_sh.worksheet('reperes')
gd.set_with_dataframe(wsheet,df_reperes)

Adding a workseet with the dame de nage data

In [45]:
import gspread
import gspread_dataframe as gd

# this piece of code assumes we already have a token to read / write a Google Sheet
# 1e_ThufABmmmbUlqQ05VoCrXfChhDuQHMnZQMYw73NxA is the id of DameDeNage_database
db_sh = gc.open_by_key('1e_ThufABmmmbUlqQ05VoCrXfChhDuQHMnZQMYw73NxA')


worksheet_data = db_sh.worksheet('data')
gd.set_with_dataframe(worksheet_data,all_ddd)